### 20190720

まずはRNA-seqから

fastqのダウンロード

SRAdbを使う

In [6]:
param <- "SRA000299"

In [1]:
library(SRAdb)

Loading required package: RSQLite
Loading required package: graph
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which.mi

In [10]:
#めちゃくちゃ時間かかるし、ファイルも重い。。。

In [5]:
sqlfile <- getSRAdbFile()

Unzipping...

Metadata associate with downloaded file:

c("schema version", "creation timestamp")c("1.0", "2019-06-27 13:35:00")


In [7]:
sra_con <- dbConnect(SQLite(), sqlfile)

In [13]:
sra_con

<SQLiteConnection>
  Path: /Users/yamada/RNA-seq/【R】トランスクリプトーム解析/chapter2/SRAmetadb.sqlite
  Extensions: TRUE

In [8]:
hoge <- sraConvert(param, sra_con=sra_con)

In [9]:
hoge

submission,study,sample,experiment,run
<chr>,<chr>,<chr>,<chr>,<chr>
SRA000299,SRP000225,SRS000561,SRX000604,SRR002322
SRA000299,SRP000225,SRS000562,SRX000605,SRR002320
SRA000299,SRP000225,SRS000561,SRX000571,SRR002321
SRA000299,SRP000225,SRS000561,SRX000571,SRR002323
SRA000299,SRP000225,SRS000562,SRX000605,SRR002325
SRA000299,SRP000225,SRS000562,SRX000606,SRR002324


In [11]:
hoge$run

[1] "SRR002322" "SRR002320" "SRR002321" "SRR002323" "SRR002325" "SRR002324"

In [12]:
k <- getFASTQinfo(hoge$run)

ERROR: Error in sraConvert(in_acc, sra_con = sra_con):  引数 "sra_con" がありませんし、省略時既定値もありません 


In [14]:
k <- getFASTQinfo(hoge$run, sra_con)

In [15]:
dim(k)

[1] 6 6

In [16]:
colnames(k)

[1] "run"        "submission" "study"      "sample"     "experiment"
[6] "ftp"

In [18]:
k

run,submission,study,sample,experiment,ftp
<chr>,<chr>,<chr>,<chr>,<chr>,<fct>
SRR002320,SRA000299,SRP000225,SRS000562,SRX000605,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002320/SRR002320.fastq.gz
SRR002321,SRA000299,SRP000225,SRS000561,SRX000571,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002321/SRR002321.fastq.gz
SRR002322,SRA000299,SRP000225,SRS000561,SRX000604,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002322/SRR002322.fastq.gz
SRR002323,SRA000299,SRP000225,SRS000561,SRX000571,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002323/SRR002323.fastq.gz
SRR002324,SRA000299,SRP000225,SRS000562,SRX000606,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002324/SRR002324.fastq.gz
SRR002325,SRA000299,SRP000225,SRS000562,SRX000605,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002325/SRR002325.fastq.gz


In [19]:
#今回は、SRR002324とSRR002322をダウンロード

In [20]:
hoge$run[c(1,6)]

[1] "SRR002322" "SRR002324"

In [22]:
getFASTQfile(hoge$run[c(1,6)], sra_con, srcType='ftp')

Files are saved to: 
'/Users/yamada/RNA-seq/【R】トランスクリプトーム解析/chapter2'



ERROR: Error in download.file(i, destfile = file.path(destDir, basename(i)), :  URL 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR002/SRR002322/SRR002322.fastq.gz' を開けません 


In [23]:
list.files(pattern=".gz")

character(0)

20190805 fastqのダウンロードから

In [7]:
getSRAfile('SRR002322', srcType='ftp', fileType='fastq')

ERROR: Error in listSRAfile(in_acc, sra_con, fileType, srcType):  引数 "sra_con" がありませんし、省略時既定値もありません 


In [5]:
getFASTQfile('SRR002322', sra_con, srcType='ftp')

ERROR: Error in dbGetQuery(sra_con, sql):  オブジェクト 'sra_con' がありません 


### リファレンス配列

In [8]:
#パッケージを読み込む
library(BSgenome)

Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:base’:

    expand.grid

Loading required package: IRanges
Loading required package: GenomeInfoDb
Loading required package: GenomicRanges
Loading required package: Biostrings
Loading required package: XVector

Attaching package: ‘Biostrings’

The following object is masked from ‘package:graph’:

    complement

The following object is masked from ‘package:base’:

    strsplit

Loading required package: rtracklayer


In [9]:
#利用可能なゲノムをリストアップ
available.genomes()

[1] "BSgenome.Alyrata.JGI.v1"                  
 [2] "BSgenome.Amellifera.BeeBase.assembly4"    
 [3] "BSgenome.Amellifera.UCSC.apiMel2"         
 [4] "BSgenome.Amellifera.UCSC.apiMel2.masked"  
 [5] "BSgenome.Aofficinalis.NCBI.V1"            
 [6] "BSgenome.Athaliana.TAIR.04232008"         
 [7] "BSgenome.Athaliana.TAIR.TAIR9"            
 [8] "BSgenome.Btaurus.UCSC.bosTau3"            
 [9] "BSgenome.Btaurus.UCSC.bosTau3.masked"     
[10] "BSgenome.Btaurus.UCSC.bosTau4"            
[11] "BSgenome.Btaurus.UCSC.bosTau4.masked"     
[12] "BSgenome.Btaurus.UCSC.bosTau6"            
[13] "BSgenome.Btaurus.UCSC.bosTau6.masked"     
[14] "BSgenome.Btaurus.UCSC.bosTau8"            
[15] "BSgenome.Carietinum.NCBI.v1"              
[16] "BSgenome.Celegans.UCSC.ce10"              
[17] "BSgenome.Celegans.UCSC.ce11"              
[18] "BSgenome.Celegans.UCSC.ce2"               
[19] "BSgenome.Celegans.UCSC.ce6"               
[20] "BSgenome.Cfamiliaris.UCSC.canFam2"        
[21] "BSgenome.Cfamiliaris.UCSC.canFam2.masked" 
[22] "BSgenome.Cfamiliaris.UCSC.canFam3"        
[23] "BSgenome.Cfamiliaris.UCSC.canFam3.masked" 
[24] "BSgenome.Dmelanogaster.UCSC.dm2"          
[25] "BSgenome.Dmelanogaster.UCSC.dm2.masked"   
[26] "BSgenome.Dmelanogaster.UCSC.dm3"          
[27] "BSgenome.Dmelanogaster.UCSC.dm3.masked"   
[28] "BSgenome.Dmelanogaster.UCSC.dm6"          
[29] "BSgenome.Drerio.UCSC.danRer10"            
[30] "BSgenome.Drerio.UCSC.danRer5"             
[31] "BSgenome.Drerio.UCSC.danRer5.masked"      
[32] "BSgenome.Drerio.UCSC.danRer6"             
[33] "BSgenome.Drerio.UCSC.danRer6.masked"      
[34] "BSgenome.Drerio.UCSC.danRer7"             
[35] "BSgenome.Drerio.UCSC.danRer7.masked"      
[36] "BSgenome.Ecoli.NCBI.20080805"             
[37] "BSgenome.Gaculeatus.UCSC.gasAcu1"         
[38] "BSgenome.Gaculeatus.UCSC.gasAcu1.masked"  
[39] "BSgenome.Ggallus.UCSC.galGal3"            
[40] "BSgenome.Ggallus.UCSC.galGal3.masked"     
[41] "BSgenome.Ggallus.UCSC.galGal4"            
[42] "BSgenome.Ggallus.UCSC.galGal4.masked"     
[43] "BSgenome.Ggallus.UCSC.galGal5"            
[44] "BSgenome.Hsapiens.1000genomes.hs37d5"     
[45] "BSgenome.Hsapiens.NCBI.GRCh38"            
[46] "BSgenome.Hsapiens.UCSC.hg17"              
[47] "BSgenome.Hsapiens.UCSC.hg17.masked"       
[48] "BSgenome.Hsapiens.UCSC.hg18"              
[49] "BSgenome.Hsapiens.UCSC.hg18.masked"       
[50] "BSgenome.Hsapiens.UCSC.hg19"              
[51] "BSgenome.Hsapiens.UCSC.hg19.masked"       
[52] "BSgenome.Hsapiens.UCSC.hg38"              
[53] "BSgenome.Hsapiens.UCSC.hg38.masked"       
[54] "BSgenome.Mfascicularis.NCBI.5.0"          
[55] "BSgenome.Mfuro.UCSC.musFur1"              
[56] "BSgenome.Mmulatta.UCSC.rheMac2"           
[57] "BSgenome.Mmulatta.UCSC.rheMac2.masked"    
[58] "BSgenome.Mmulatta.UCSC.rheMac3"           
[59] "BSgenome.Mmulatta.UCSC.rheMac3.masked"    
[60] "BSgenome.Mmulatta.UCSC.rheMac8"           
[61] "BSgenome.Mmusculus.UCSC.mm10"             
[62] "BSgenome.Mmusculus.UCSC.mm10.masked"      
[63] "BSgenome.Mmusculus.UCSC.mm8"              
[64] "BSgenome.Mmusculus.UCSC.mm8.masked"       
[65] "BSgenome.Mmusculus.UCSC.mm9"              
[66] "BSgenome.Mmusculus.UCSC.mm9.masked"       
[67] "BSgenome.Osativa.MSU.MSU7"                
[68] "BSgenome.Ptroglodytes.UCSC.panTro2"       
[69] "BSgenome.Ptroglodytes.UCSC.panTro2.masked"
[70] "BSgenome.Ptroglodytes.UCSC.panTro3"       
[71] "BSgenome.Ptroglodytes.UCSC.panTro3.masked"
[72] "BSgenome.Ptroglodytes.UCSC.panTro5"       
[73] "BSgenome.Ptroglodytes.UCSC.panTro6"       
[74] "BSgenome.Rnorvegicus.UCSC.rn4"            
[75] "BSgenome.Rnorvegicus.UCSC.rn4.masked"     
[76] "BSgenome.Rnorvegicus.UCSC.rn5"            
[77] "BSgenome.Rnorvegicus.UCSC.rn5.masked"     
[78] "BSgenome.Rnorvegicus.UCSC.rn6"            
[79] "BSgenome.Scerevisiae.UCSC.sacCer1"        
[80] "BSgenome.Scerevisiae.UCSC.sacCer2"        
[81] "BSgenome.Scerevisiae.UCSC.sacCer3"        
[82] "BSgenome.Sscrofa.UCSC.susS

In [10]:
hoge <- available.genomes(splitNameParts=T)

In [11]:
dim(hoge)

[1] 91  5

In [12]:
colnames(hoge)

[1] "pkgname"          "organism"         "provider"         "provider_version"
[5] "masked"

In [14]:
hoge$organism

[1] Alyrata       Amellifera    Amellifera    Amellifera    Aofficinalis 
 [6] Athaliana     Athaliana     Btaurus       Btaurus       Btaurus      
[11] Btaurus       Btaurus       Btaurus       Btaurus       Carietinum   
[16] Celegans      Celegans      Celegans      Celegans      Cfamiliaris  
[21] Cfamiliaris   Cfamiliaris   Cfamiliaris   Dmelanogaster Dmelanogaster
[26] Dmelanogaster Dmelanogaster Dmelanogaster Drerio        Drerio       
[31] Drerio        Drerio        Drerio        Drerio        Drerio       
[36] Ecoli         Gaculeatus    Gaculeatus    Ggallus       Ggallus      
[41] Ggallus       Ggallus       Ggallus       Hsapiens      Hsapiens     
[46] Hsapiens      Hsapiens      Hsapiens      Hsapiens      Hsapiens     
[51] Hsapiens      Hsapiens      Hsapiens      Mfascicularis Mfuro        
[56] Mmulatta      Mmulatta      Mmulatta      Mmulatta      Mmulatta     
[61] Mmusculus     Mmusculus     Mmusculus     Mmusculus     Mmusculus    
[66] Mmusculus     Osativa       Ptroglodytes  Ptroglodytes  Ptroglodytes 
[71] Ptroglodytes  Ptroglodytes  Ptroglodytes  Rnorvegicus   Rnorvegicus  
[76] Rnorvegicus   Rnorvegicus   Rnorvegicus   Scerevisiae   Scerevisiae  
[81] Scerevisiae   Sscrofa       Sscrofa       Sscrofa       Tgondii      
[86] Tguttata      Tguttata      Tguttata      Vvinifera     Vvinifera    
[91] Vvinifera    
26 Levels: Alyrata Amellifera Aofficinalis Athaliana Btaurus ... Vvinifera

In [15]:
#生物種ごとの種類のテーブル
table(hoge$organism)


      Alyrata    Amellifera  Aofficinalis     Athaliana       Btaurus 
            1             3             1             2             7 
   Carietinum      Celegans   Cfamiliaris Dmelanogaster        Drerio 
            1             4             4             5             7 
        Ecoli    Gaculeatus       Ggallus      Hsapiens Mfascicularis 
            1             2             5            10             1 
        Mfuro      Mmulatta     Mmusculus       Osativa  Ptroglodytes 
            1             5             6             1             6 
  Rnorvegicus   Scerevisiae       Sscrofa       Tgondii      Tguttata 
            5             3             3             1             3 
    Vvinifera 
            3 

In [16]:
#インストール済みのパッケージを調べる
installed.genomes()

character(0)

#### humanのリファレンスをダウンロード

In [17]:
param <- "BSgenome.Hsapiens.UCSC.hg38"

In [18]:
source("http://bioconductor.org/biocLite.R")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help
A newer version of Bioconductor is available for this version of R,
  ?BiocUpgrade for help


In [19]:
biocLite(param)

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Installing package(s) ‘BSgenome.Hsapiens.UCSC.hg38’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'blob', 'boot', 'callr', 'checkmate', 'class', 'cli', 'clipr',
  'cluster', 'codetools', 'curl', 'dbplyr', 'devtools', 'digest', 'dplyr',
  'evaluate', 'foreign', 'formatR', 'ggplot2', 'highr', 'Hmisc', 'hms',
  'htmlTable', 'htmlwidgets', 'httpuv', 'httr', 'IRkernel', 'knitr', 'later',
  'markdown', 'Matrix', 'mgcv', 'nlme', 'openssl', 'pillar', 'pkgbuild',
  'pkgload', 'processx', 'purrr', 'rcmdcheck', 'Rcpp', 'RcppArmadillo',
  'RCurl', 'remotes', 'rpart', 'RSQLite', 'rstudioapi', 'scales', 'shiny',
  'stringi', 'stringr', 'survival', 'testthat', 'tibble', 'tidyr',
  'tidyselect', 'usethis', 'vctrs', 'XML', 'xml2', 'xtable'


In [21]:
#インストール済みのパッケージを再度調べる
installed.genomes()

[1] "BSgenome.Hsapiens.UCSC.hg38"

#### トランスクリプトーム配列を取得

In [22]:
out_f <- "hoge7.txt"  #出力ファイル名を指定してout_fに収納
param_dataset <- "hsapiens_gene_ensembl"#データセット名を指定
param_attribute <- "refseq_mrna"       #配列を取得したい属性名を指定
param_seqtype <- "cdna"     #配列のタイプを指定

In [23]:
library(biomaRt)

In [24]:
#データベース名("ensembl")とparam_datasetで指定したデータセット名を与えてmartに格納
mart <- useMart("ensembl",dataset=param_dataset)

In [25]:
#指定したIDリスト取得結果をhogeに格納
hoge <- getBM(attributes=param_attribute, filters="", values="", mart=mart)

In [26]:
#指定したパラメータで配列取得した結果をoutに格納
out <- getSequence(id=hoge, type=param_attribute,
                seqType=param_seqtype, mart=mart)

In [27]:
#outの中身を指定したファイル名で保存
exportFASTA(out, file=out_f)

行列は dim 属性という次元の属性を持っており，(行数 ，列数) という長さ 2 の整数ベクトルの形をしている

In [28]:
#配列数を表示
dim(out)

[1] 41338     2

ヒト以外の生物種の情報を取得可能か調べる

In [29]:
hoge <- listDatasets(useMart("ensembl"))

In [30]:
dim(hoge)

[1] 190   3

In [31]:
head(hoge)

dataset,description,version
<I<chr>>,<I<chr>>,<I<chr>>
abrachyrhynchus_gene_ensembl,Pink-footed goose genes (ASM259213v1),ASM259213v1
acalliptera_gene_ensembl,Eastern happy genes (fAstCal1.2),fAstCal1.2
acarolinensis_gene_ensembl,Anole lizard genes (AnoCar2.0),AnoCar2.0
acitrinellus_gene_ensembl,Midas cichlid genes (Midas_v5),Midas_v5
ahaastii_gene_ensembl,Great spotted kiwi genes (aptHaa1),aptHaa1
amelanoleuca_gene_ensembl,Panda genes (ailMel1),ailMel1


### アノテーション情報を入手  
http://www.iu.a.u-tokyo.ac.jp/~kadota/r_seq.html#intro_ngs_annotation_biomart

In [32]:
#出力ファイル名を指定してout_fに格納
out_f <- "hoge9.txt"
#データセット名を指定
param_dataset <- "hsapiens_gene_ensembl"
#取得したい属性名を指定
param_attribute <- c("refseq_mrna", "hgnc_symbol", "go_id") 
#フィルタリング条件を指定
param_filter <- "with_ox_refseq_mrna"

In [33]:
#前処理（取得可能なアノテーション情報をリストアップ）

#データベース名("ensembl")とparam_datasetで指定したデータセット名を与えてmartに格納
mart <- useMart("ensembl",dataset=param_dataset)

In [44]:
#アノテーション情報取得結果をhogeに格納
out <- getBM(attributes=param_attribute, filters=param_filter, values=list(TRUE), mart=mart)

ERROR: Error in getBM(attributes = param_attribute, filters = param_filter, values = list(TRUE), : Invalid filters(s): with_ox_refseq_mrna 
Please use the function 'listFilters' to get valid filter names


In [39]:
#エラーどうり確認してみる
listFilters(mart)

name,description
<chr>,<chr>
chromosome_name,Chromosome/scaffold name
start,Start
end,End
band_start,Band Start
band_end,Band End
marker_start,Marker Start
marker_end,Marker End
encode_region,Encode region
strand,Strand


In [38]:
listFilters(mart)$name

[1] "chromosome_name"                      
  [2] "start"                                
  [3] "end"                                  
  [4] "band_start"                           
  [5] "band_end"                             
  [6] "marker_start"                         
  [7] "marker_end"                           
  [8] "encode_region"                        
  [9] "strand"                               
 [10] "chromosomal_region"                   
 [11] "with_ccds"                            
 [12] "with_chembl"                          
 [13] "with_clone_based_ensembl_gene"        
 [14] "with_clone_based_ensembl_transcript"  
 [15] "with_dbass3"                          
 [16] "with_dbass5"                          
 [17] "with_ens_hs_transcript"               
 [18] "with_ens_hs_translation"              
 [19] "with_entrezgene_trans_name"           
 [20] "with_embl"                            
 [21] "with_arrayexpress"                    
 [22] "with_genedb"                          
 [23] "with_go"                              
 [24] "with_goslim_goa"                      
 [25] "with_hgnc"                            
 [26] "with_hgnc_trans_name"                 
 [27] "with_hpa"                             
 [28] "with_protein_id"                      
 [29] "with_kegg_enzyme"                     
 [30] "with_ens_lrg_gene"                    
 [31] "with_ens_lrg_transcript"              
 [32] "with_merops"                          
 [33] "with_mim_gene"                        
 [34] "with_mim_morbid"                      
 [35] "with_mirbase"                         
 [36] "with_mirbase_trans_name"              
 [37] "with_entrezgene"                      
 [38] "with_pdb"                             
 [39] "with_reactome"                        
 [40] "with_reactome_gene"                   
 [41] "with_reactome_transcript"             
 [42] "with_refseq_mrna"                     
 [43] "with_refseq_mrna_predicted"           
 [44] "with_refseq_ncrna"                    
 [45] "with_refseq_ncrna_predicted"          
 [46] "with_refseq_peptide"                  
 [47] "with_refseq_peptide_predicted"        
 [48] "with_rfam"                            
 [49] "with_rfam_trans_name"                 
 [50] "with_rnacentral"                      
 [51] "with_ucsc"                            
 [52] "with_uniparc"                         
 [53] "with_uniprot_gn"                      
 [54] "with_uniprotswissprot"                
 [55] "with_uniprotsptrembl"                 
 [56] "with_wikigene"                        
 [57] "ensembl_gene_id"                      
 [58] "ensembl_gene_id_version"              
 [59] "ensembl_transcript_id"                
 [60] "ensembl_transcript_id_version"        
 [61] "ensembl_peptide_id"                   
 [62] "ensembl_peptide_id_version"           
 [63] "ensembl_exon_id"                      
 [64] "external_gene_name"                   
 [65] "external_transcript_name"             
 [66] "ccds"                                 
 [67] "chembl"                               
 [68] "clone_based_ensembl_gene"             
 [69] "clone_based_ensembl_transcript"       
 [70] "dbass3_name"                          
 [71] "dbass3_id"                            
 [72] "dbass5_name"                          
 [73] "dbass5_id"                            
 [74] "ens_hs_transcript"                    
 [75] "ens_hs_translation"                   
 [76] "entrezgene_trans_name"                
 [77] "embl"                                 
 [78] "arrayexpress"                         
 [79] "genedb"                               
 [80] "go"                                   
 [81] "goslim_goa"                           
 [82] "hgnc_id"                              
 [83] "hgnc_symbol"                          
 [84] "hgnc_trans_name"                      
 [85] "hpa_accession"                        
 [86] "hpa_id"                               
 [87] "protein_id"                           


refseq_mrnaでやってみる

In [45]:
#アノテーション情報取得結果をhogeに格納
out <- getBM(attributes=param_attribute, filters="refseq_mrna", values=list(TRUE), mart=mart)

In [46]:
head(out)

refseq_mrna,hgnc_symbol,go_id
<lgl>,<lgl>,<lgl>


In [47]:
#outの中身が何もない

In [41]:
#ファイルに保存

#outの中身を指定したファイル名で保存
write.table(out, out_f, sep="\t", append=F, quote=F, row.names=F)

代わりにこれでやってみる

In [51]:
out <- getBM(attributes=param_attribute, filters="", values="", mart=mart)

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): transfer closed with outstanding read data remaining


In [49]:
#outの確認
out

refseq_mrna,hgnc_symbol,go_id
<lgl>,<lgl>,<lgl>


In [52]:
dim(out)

[1] 0 3

In [ ]:
out[1:15,]

In [53]:
dim(listFilters(mart))

[1] 434   2

In [54]:
head(listFilters(mart))

name,description
<chr>,<chr>
chromosome_name,Chromosome/scaffold name
start,Start
end,End
band_start,Band Start
band_end,Band End
marker_start,Marker Start


In [55]:
listFilters(mart)[c(97,137),]

,name,description
,<chr>,<chr>
97,entrezgene_accession,NCBI gene accession(s) [e.g. A1BG]
137,with_affy_primeview,With AFFY PrimeView probe ID(s)


In [58]:
listFilters(mart)[103,]

,name,description
,<chr>,<chr>
103,refseq_mrna,RefSeq mRNA ID(s) [e.g. NM_000014]


In [60]:
out <- getBM(attributes=param_attribute, mart=mart)

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): transfer closed with outstanding read data remaining


In [ ]:
#行列outの1列目の情報が存在するもののみフィルタリング
out <- subset(out, refseq_mrna != "")

In [ ]:
dim(out)

In [ ]:
out[1:10,]

### 20190812

### 2.3.4　マッピング（準備）

In [1]:
enkichikan <- function(fa, p) {        #関数名や引数の作成
    t <- substring(fa, p, p)           #置換したい位置の塩基を取りだす
    t_c <- chartr("CGAT", "GCTA", t)   #置換後の塩基を作成
    substring(fa, p, p) <- t_c         #置換
    return(fa)                         #置換後のデータを返す
}

In [2]:
enkichikan("CCGTACG", 2)

[1] "CGGTACG"

参考：https://stats.biopapyrus.jp/r/basic/string.html

In [ ]:
#部分文字列の切り出し
substring(文字列 , 要素の最初の位置, 要素の最後の位置) 

In [16]:
#chartr は一文字ずつの置換を行う関数

In [3]:
out_f <- "ref_genome.fa"               #出力ファイル名を指定してout_fに格納
param1 <- c(48, 160, 100, 123)         #配列長を指定
narabi <- c("A","C","G","T")           #ACGTの並びを指定
param2 <- c(28, 22, 26, 24)            #(A,C,G,Tの並びで)各塩基の存在比率を指定
param3 <- "chr"                        #FASTA形式ファイルのdescription部分を指定
param4 <- 3                            #コピーを作成したい配列番号を指定
param5 <- c(2, 7)                      #コピー先配列の塩基置換したい位置を指定

In [1]:
library(Biostrings)

Loading required package: BiocGenerics
Loading required package: parallel
Warning message in sample.int(.Machine$integer.max - 1L, 1L):
“ '.Random.seed[1]' が有効な整数値ではないため、無視されました ”
Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,


In [5]:
#chr1-4の生成
set.seed(1000)                         #おまじない(同じ乱数になるようにするため)
ACGTset <- rep(narabi, param2)         #文字列ベクトルACGTsetを作成
hoge <- NULL                           #hogeというプレースホルダの作成

Warning message in set.seed(1000):
“ '.Random.seed[1]' が有効な整数値ではないため、無視されました ”

In [17]:
ACGTset

[1] "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "A"
 [19] "A" "A" "A" "A" "A" "A" "A" "A" "A" "A" "C" "C" "C" "C" "C" "C" "C" "C"
 [37] "C" "C" "C" "C" "C" "C" "C" "C" "C" "C" "C" "C" "C" "C" "G" "G" "G" "G"
 [55] "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G" "G"
 [73] "G" "G" "G" "G" "T" "T" "T" "T" "T" "T" "T" "T" "T" "T" "T" "T" "T" "T"
 [91] "T" "T" "T" "T" "T" "T" "T" "T" "T" "T"

In [6]:
#length(param1)で表現される配列数分だけループを回す
#ACGTsetの文字型ベクトルからparam1[i]回分だけ復元抽出して得られた塩基配列をhogeに格納
for(i in 1:length(param1)){
    hoge <- c(hoge, paste(sample(ACGTset, param1[i], replace=T), collapse=""))
}

In [18]:
length(param1)

[1] 4

In [20]:
hoge

[1] "CGAGGAGGAACGCTTACGAGATCAGGCTAAGAGTGGATGCTGAGTGGG"                                                                                                                
[2] "AGGGAGGGGGTCCAGTATCTATGGCCTAAAAACATAGACACCTTGAGGAGACGCAGGTAGGCTGAGGATAAAGCCGTTTGCACGCATCATGAAGGGGCTGCTCGGGTATGGTTAGTCTTTGCCTCTAGATTTTCACGACGCTGCGGTTCATGACGCCCTG"
[3] "GGGGGGACTATTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGGCAGCATCTAGTCGCATCAGAAGGGTGTAGTCAGCCTATAGTTAACTAGTTT"                                                            
[4] "CGAGACGAGCAAGTTATTCGCTCAGTGAATGGGTAGCAAAAGAATGTTGTCGTCTGTATTGGGGCCTATGCTCGACAAGAGATTGTGTGTAGTATGAGCCACCAGACTTTACCGTACAAGATA"                                     
[5] "GCGGGGTCTATTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGGCAGCATCTAGTCGCATCAGAAGGGTGTAGTCAGCCTATAGTTAACTAGTTT"

In [21]:
hoge[5]

[1] "GCGGGGTCTATTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGGCAGCATCTAGTCGCATCAGAAGGGTGTAGTCAGCCTATAGTTAACTAGTTT"

In [7]:
#chr5の生成と塩基置換導入

#(param4)番目の配列をコピーして追加
hoge <- c(hoge, hoge[param4])
#塩基置換
hoge[length(param1)+1] <- enkichikan(hoge[length(param1)+1], param5[1])
hoge[length(param1)+1] <- enkichikan(hoge[length(param1)+1], param5[2])

In [8]:
#ファイルに保存

#DNAStringSetオブジェクトに変更
fasta <- DNAStringSet(hoge)
#description行を追加
names(fasta) <- paste(param3, 1:length(hoge), sep="")
#fastaの中身を指定したファイル名で保存
writeXStringSet(fasta, file=out_f, format="fasta", width=50)

### p82

In [1]:
list.files()

[1] "20190720_chapter2.ipynb" "hoge7.txt"              
[3] "list_sub3.txt"           "mapping"                
[5] "mapping_paired.txt"      "mapping_single2.txt"

In [10]:
fasta

  A DNAStringSet instance of length 5
    width seq                                               names               
[1]    48 CGAGGAGGAACGCTTACGAGATCAGGCTAAGAGTGGATGCTGAGTGGG  chr1
[2]   160 AGGGAGGGGGTCCAGTATCTATG...ACGCTGCGGTTCATGACGCCCTG chr2
[3]   100 GGGGGGACTATTTCCCCGCTTGC...GTCAGCCTATAGTTAACTAGTTT chr3
[4]   123 CGAGACGAGCAAGTTATTCGCTC...ACCAGACTTTACCGTACAAGATA chr4
[5]   100 GCGGGGTCTATTTCCCCGCTTGC...GTCAGCCTATAGTTAACTAGTTT chr5

In [11]:
width(fasta)

[1]  48 160 100 123 100

In [13]:
names(fasta)

[1] "chr1" "chr2" "chr3" "chr4" "chr5"

In [14]:
fasta[5]

  A DNAStringSet instance of length 1
    width seq                                               names               
[1]   100 GCGGGGTCTATTTCCCCGCTTGC...GTCAGCCTATAGTTAACTAGTTT chr5

### RNA-seqリード配列（マップする側）の作成

In [2]:
DNAString_chartr <- function(fa, p) {  #関数名や引数の作成
  str_list <- as.character(fa)         #文字列に変更
  t <- substring(str_list, p, p)       #置換したい位置の塩基を取りだす
  t_c <- chartr("CGAT", "GCTA", t)     #置換後の塩基を作成
  substring(str_list, p, p) <- t_c     #置換
  fa_r <- DNAStringSet(str_list)       #DNAStringSetオブジェクトに戻す
  names(fa_r) <- names(fa)             #description部分の情報を追加
  return(fa_r)                         #置換後のデータを返す
}

In [3]:
in_f1 <- "ref_genome.fa"               #入力ファイル名を指定してin_f1に格納(multi-FASTAファイル)
in_f2 <- "list_sub3.txt"               #入力ファイル名を指定してin_f2に格納(リストファイル)
out_f <- "sample_RNAseq1.fa"           #出力ファイル名を指定してout_fに格納
param <- 4                             #最後のリードの塩基置換したい位置を指定

In [4]:
#入力ファイルの読み込み
fasta <- readDNAStringSet(in_f1, format="fasta")#in_f1で指定したファイルの読み込み
posi <- read.table(in_f2)              #in_f2で指定したファイルの読み込み

In [5]:
#部分配列の抽出および最後のリードの塩基置換

#最終的に得る結果を格納するためのプレースホルダhogeを作成しているだけ
hoge <- NULL

#length(posi)回だけループを回す
#条件を満たすかどうかを判定した結果をobjに格納
#subseq関数を用いてobjがTRUEとなるもののみに対して、posi[i,2]とposi[i,3]で与えた範囲に対応する部分配列を抽出した結果をhogeに格納

for(i in 1:nrow(posi)){                
    obj <- names(fasta) == posi[i,1]   
    hoge <- append(hoge, subseq(fasta[obj], start=posi[i,2], end=posi[i,3]))
}

#hogeの中身をfastaに格納
fasta <- hoge                      
#指定した位置の塩基置換を実行した結果をfastaに格納
fasta[nrow(posi)] <- DNAString_chartr(fasta[nrow(posi)], param)

In [6]:
fasta

  A DNAStringSet instance of length 8
    width seq                                               names               
[1]    35 CGCTTACGAGATCAGGCTAAGAGTGGATGCTGAGT               chr1
[2]    35 TATCTATGGCCTAAAAACATAGACACCTTGAGGAG               chr2
[3]    35 AGGGAGGGGGTCCAGTATCTATGGCCTAAAAACAT               chr2
[4]    35 TTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATA               chr3
[5]    35 CCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGG               chr3
[6]    35 GGGGACTATTTCCCCGCTTGCAGGAATCGTGTCAG               chr3
[7]    35 GGGGGGACTATTTCCCCGCTTGCAGGAATCGTGTC               chr3
[8]    35 GCGCGGTCTATTTCCCCGCTTGCAGGAATCGTGTC               chr5

In [7]:
hoge

  A DNAStringSet instance of length 8
    width seq                                               names               
[1]    35 CGCTTACGAGATCAGGCTAAGAGTGGATGCTGAGT               chr1
[2]    35 TATCTATGGCCTAAAAACATAGACACCTTGAGGAG               chr2
[3]    35 AGGGAGGGGGTCCAGTATCTATGGCCTAAAAACAT               chr2
[4]    35 TTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATA               chr3
[5]    35 CCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGG               chr3
[6]    35 GGGGACTATTTCCCCGCTTGCAGGAATCGTGTCAG               chr3
[7]    35 GGGGGGACTATTTCCCCGCTTGCAGGAATCGTGTC               chr3
[8]    35 GCGGGGTCTATTTCCCCGCTTGCAGGAATCGTGTC               chr5

In [8]:
#ファイルに保存

#行列posiの各列を"_"で結合したものをdescriptionに格納
description <- paste(posi[,1], posi[,2], posi[,3], sep="_")
#description行に相当する記述を追加している
names(fasta) <- description
#fastaの中身を指定したファイル名で保存
writeXStringSet(fasta, file=out_f, format="fasta", width=50)

In [10]:
posi

V1,V2,V3
<fct>,<int>,<int>
chr1,11,45
chr2,16,50
chr2,1,35
chr3,11,45
chr3,15,49
chr3,3,37
chr3,1,35
chr5,1,35


In [9]:
dim(posi)

[1] 8 3

In [11]:
list.files()

[1] "20190720_chapter2.ipynb" "hoge7.txt"              
[3] "list_sub3.txt"           "ref_genome.fa"          
[5] "sample_RNAseq1.fa"

In [12]:
fasta

  A DNAStringSet instance of length 8
    width seq                                               names               
[1]    35 CGCTTACGAGATCAGGCTAAGAGTGGATGCTGAGT               chr1_11_45
[2]    35 TATCTATGGCCTAAAAACATAGACACCTTGAGGAG               chr2_16_50
[3]    35 AGGGAGGGGGTCCAGTATCTATGGCCTAAAAACAT               chr2_1_35
[4]    35 TTTCCCCGCTTGCAGGAATCGTGTCAGTTGGTATA               chr3_11_45
[5]    35 CCCGCTTGCAGGAATCGTGTCAGTTGGTATACAGG               chr3_15_49
[6]    35 GGGGACTATTTCCCCGCTTGCAGGAATCGTGTCAG               chr3_3_37
[7]    35 GGGGGGACTATTTCCCCGCTTGCAGGAATCGTGTC               chr3_1_35
[8]    35 GCGCGGTCTATTTCCCCGCTTGCAGGAATCGTGTC               chr5_1_35

#### リストファイルの作成

In [13]:
out_f <- "mapping_single1.txt"         #出力ファイル名を指定してout_fに格納
param_FN <- "sample_RNAseq1.fa"        #マップしたいファイル名を指定
param_SN <- "hoge1"                    #任意のサンプル名を指定
out <- cbind(param_FN, param_SN)
colnames(out) <- c("FileName", "SampleName")
write.table(out, out_f, sep="\t", append=F, quote=F, row.names=F)

In [14]:
out

FileName,SampleName
sample_RNAseq1.fa,hoge1


In [18]:
#複数サンプルver

In [15]:
out_f <- "mapping_single2.txt"         #出力ファイル名を指定してout_fに格納
param_FN <- c("sample_RNAseq1.fa", "sample_RNAseq2.fa")#マップしたいファイル名を指定
param_SN <- c("sample1", "sample2")    #任意のサンプル名を指定
out <- cbind(param_FN, param_SN)
colnames(out) <- c("FileName", "SampleName")
write.table(out, out_f, sep="\t", append=F, quote=F, row.names=F)

In [16]:
out

FileName,SampleName
sample_RNAseq1.fa,sample1
sample_RNAseq2.fa,sample2


In [19]:
#pair-end ver

In [20]:
out_f <- "mapping_paired.txt"          #出力ファイル名を指定してout_fに格納
param_FN1 <- "sample_paired1.fq.gz"
param_FN2 <- "sample_paired2.fq.gz"
param_SN <- "uge"
out <- cbind(param_FN1, param_FN2, param_SN)
colnames(out) <- c("FileName1", "FileName2", "SampleName")
write.table(out, out_f, sep="\t", append=F, quote=F, row.names=F)

In [21]:
out

FileName1,FileName2,SampleName
sample_paired1.fq.gz,sample_paired2.fq.gz,uge


### マッピング（本番）

In [25]:
#ディレクトリを移動する

In [2]:
setwd('./mapping')

In [3]:
list.files()

[1] "mapping_single1.txt"       "QuasR_log_51d341a1bec.txt"
[3] "QuasR_log_51d38bebb5d.txt" "ref_genome.fa"            
[5] "ref_genome.fa.fai"         "ref_genome.fa.md5"        
[7] "ref_genome.fa.Rbowtie"     "sample_RNAseq1.fa"

In [4]:
#入力ファイル名を指定してin_f1に格納(RNA-seqファイル)
in_f1 <- "mapping_single1.txt"
#入力ファイル名を指定してin_f2に格納(リファレンス配列)
in_f2 <- "ref_genome.fa"

In [8]:
#パッケージの読み込み
library(QuasR)

Loading required package: parallel
Warning message in sample.int(.Machine$integer.max - 1L, 1L):
“ '.Random.seed[1]' が有効な整数値ではないため、無視されました ”Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, row

In [9]:
#マッピングを行うqAlign関数を実行した結果をoutに格納
out <- qAlign(in_f1, in_f2, splicedAlignment=F)

alignment files missing - need to:
    create 1 genomic alignment(s)
Testing the compute nodes...OK
Loading QuasR on the compute nodes...OK
Available cores:


nodeNames
kyamadaPro 
         1 


Performing genomic alignments for 1 samples. See progress in the log file:
/Users/yamada/RNA-seq/【R】トランスクリプトーム解析/chapter2/mapping/QuasR_log_2ffd158bd04d.txt


ERROR: Error in checkForRemoteErrors(val): one node produced an error: Error on kyamadaPro processing sample /Users/yamada/RNA-seq/【R】トランスクリプトーム解析/chapter2/mapping/sample_RNAseq1.fa : 'asBam' truncated input file at record 2
  SAM file: '/var/folders/8g/l6tj9lds0z3b15_dc_wgvy9w0000gn/T//Rtmp33D540/sample_RNAseq1.fa46a135354f1c.sam'



In [29]:
list.files()

[1] "mapping_single1.txt"       "QuasR_log_51d38bebb5d.txt"
[3] "ref_genome.fa"             "ref_genome.fa.fai"        
[5] "ref_genome.fa.md5"         "ref_genome.fa.Rbowtie"    
[7] "sample_RNAseq1.fa"

### マッピング　本番　実際のデータ

In [43]:
setwd('./human')

In [44]:
list.files()

[1] "mapping_single3.txt" "SRR002322.sra"       "SRR002322.sra.fastq"
[4] "SRR002324.sra"       "SRR002324.sra.fastq"

#### mapping_single3.txtの作成

In [14]:
out_f <- "mapping_single3.txt"         #出力ファイル名を指定してout_fに格納
param_FN <- c("SRR002324.fastq.gz", "SRR002322.fastq.gz")#マップしたいファイル名を指定
param_SN <- c("Kidney", "Liver")    #任意のサンプル名を指定
out <- cbind(param_FN, param_SN)
colnames(out) <- c("FileName", "SampleName")
write.table(out, out_f, sep="\t", append=F, quote=F, row.names=F)

In [15]:
out

FileName,SampleName
SRR002324.fastq.gz,Kidney
SRR002322.fastq.gz,Liver


リファレンス配列について

In [16]:
library(BSgenome)
available.genomes() 

Loading required package: Biostrings
Loading required package: XVector

Attaching package: ‘Biostrings’

The following object is masked from ‘package:base’:

    strsplit

Loading required package: rtracklayer


[1] "BSgenome.Alyrata.JGI.v1"                  
 [2] "BSgenome.Amellifera.BeeBase.assembly4"    
 [3] "BSgenome.Amellifera.UCSC.apiMel2"         
 [4] "BSgenome.Amellifera.UCSC.apiMel2.masked"  
 [5] "BSgenome.Aofficinalis.NCBI.V1"            
 [6] "BSgenome.Athaliana.TAIR.04232008"         
 [7] "BSgenome.Athaliana.TAIR.TAIR9"            
 [8] "BSgenome.Btaurus.UCSC.bosTau3"            
 [9] "BSgenome.Btaurus.UCSC.bosTau3.masked"     
[10] "BSgenome.Btaurus.UCSC.bosTau4"            
[11] "BSgenome.Btaurus.UCSC.bosTau4.masked"     
[12] "BSgenome.Btaurus.UCSC.bosTau6"            
[13] "BSgenome.Btaurus.UCSC.bosTau6.masked"     
[14] "BSgenome.Btaurus.UCSC.bosTau8"            
[15] "BSgenome.Carietinum.NCBI.v1"              
[16] "BSgenome.Celegans.UCSC.ce10"              
[17] "BSgenome.Celegans.UCSC.ce11"              
[18] "BSgenome.Celegans.UCSC.ce2"               
[19] "BSgenome.Celegans.UCSC.ce6"               
[20] "BSgenome.Cfamiliaris.UCSC.canFam2"        
[21] "BSgenome.Cfamiliaris.UCSC.canFam2.masked" 
[22] "BSgenome.Cfamiliaris.UCSC.canFam3"        
[23] "BSgenome.Cfamiliaris.UCSC.canFam3.masked" 
[24] "BSgenome.Dmelanogaster.UCSC.dm2"          
[25] "BSgenome.Dmelanogaster.UCSC.dm2.masked"   
[26] "BSgenome.Dmelanogaster.UCSC.dm3"          
[27] "BSgenome.Dmelanogaster.UCSC.dm3.masked"   
[28] "BSgenome.Dmelanogaster.UCSC.dm6"          
[29] "BSgenome.Drerio.UCSC.danRer10"            
[30] "BSgenome.Drerio.UCSC.danRer5"             
[31] "BSgenome.Drerio.UCSC.danRer5.masked"      
[32] "BSgenome.Drerio.UCSC.danRer6"             
[33] "BSgenome.Drerio.UCSC.danRer6.masked"      
[34] "BSgenome.Drerio.UCSC.danRer7"             
[35] "BSgenome.Drerio.UCSC.danRer7.masked"      
[36] "BSgenome.Ecoli.NCBI.20080805"             
[37] "BSgenome.Gaculeatus.UCSC.gasAcu1"         
[38] "BSgenome.Gaculeatus.UCSC.gasAcu1.masked"  
[39] "BSgenome.Ggallus.UCSC.galGal3"            
[40] "BSgenome.Ggallus.UCSC.galGal3.masked"     
[41] "BSgenome.Ggallus.UCSC.galGal4"            
[42] "BSgenome.Ggallus.UCSC.galGal4.masked"     
[43] "BSgenome.Ggallus.UCSC.galGal5"            
[44] "BSgenome.Hsapiens.1000genomes.hs37d5"     
[45] "BSgenome.Hsapiens.NCBI.GRCh38"            
[46] "BSgenome.Hsapiens.UCSC.hg17"              
[47] "BSgenome.Hsapiens.UCSC.hg17.masked"       
[48] "BSgenome.Hsapiens.UCSC.hg18"              
[49] "BSgenome.Hsapiens.UCSC.hg18.masked"       
[50] "BSgenome.Hsapiens.UCSC.hg19"              
[51] "BSgenome.Hsapiens.UCSC.hg19.masked"       
[52] "BSgenome.Hsapiens.UCSC.hg38"              
[53] "BSgenome.Hsapiens.UCSC.hg38.masked"       
[54] "BSgenome.Mfascicularis.NCBI.5.0"          
[55] "BSgenome.Mfuro.UCSC.musFur1"              
[56] "BSgenome.Mmulatta.UCSC.rheMac2"           
[57] "BSgenome.Mmulatta.UCSC.rheMac2.masked"    
[58] "BSgenome.Mmulatta.UCSC.rheMac3"           
[59] "BSgenome.Mmulatta.UCSC.rheMac3.masked"    
[60] "BSgenome.Mmulatta.UCSC.rheMac8"           
[61] "BSgenome.Mmusculus.UCSC.mm10"             
[62] "BSgenome.Mmusculus.UCSC.mm10.masked"      
[63] "BSgenome.Mmusculus.UCSC.mm8"              
[64] "BSgenome.Mmusculus.UCSC.mm8.masked"       
[65] "BSgenome.Mmusculus.UCSC.mm9"              
[66] "BSgenome.Mmusculus.UCSC.mm9.masked"       
[67] "BSgenome.Osativa.MSU.MSU7"                
[68] "BSgenome.Ptroglodytes.UCSC.panTro2"       
[69] "BSgenome.Ptroglodytes.UCSC.panTro2.masked"
[70] "BSgenome.Ptroglodytes.UCSC.panTro3"       
[71] "BSgenome.Ptroglodytes.UCSC.panTro3.masked"
[72] "BSgenome.Ptroglodytes.UCSC.panTro5"       
[73] "BSgenome.Ptroglodytes.UCSC.panTro6"       
[74] "BSgenome.Rnorvegicus.UCSC.rn4"            
[75] "BSgenome.Rnorvegicus.UCSC.rn4.masked"     
[76] "BSgenome.Rnorvegicus.UCSC.rn5"            
[77] "BSgenome.Rnorvegicus.UCSC.rn5.masked"     
[78] "BSgenome.Rnorvegicus.UCSC.rn6"            
[79] "BSgenome.Scerevisiae.UCSC.sacCer1"        
[80] "BSgenome.Scerevisiae.UCSC.sacCer2"        
[81] "BSgenome.Scerevisiae.UCSC.sacCer3"        
[82] "BSgenome.Sscrofa.UCSC.susS

In [17]:
installed.genomes() 

[1] "BSgenome.Hsapiens.UCSC.hg38"

任意のリファレンス配列のダウンロード

In [ ]:
param <- "BSgenome.Tgondii.ToxoDB.7.0" #インストールしたいパッケージ名を指定
source("http://bioconductor.org/biocLite.R")
biocLite(param)

#### マッピングの実行

In [46]:
in_f1 <- "mapping_single3.txt"         #入力ファイル名を指定してin_f1に格納(RNA-seqファイル)
in_f2 <- "BSgenome.Hsapiens.UCSC.hg38" #入力ファイル名を指定してin_f2に格納(リファレンス配列)
param <- "-m 1 --best --strata -v 2"   #マッピング時のオプションを指定

In [45]:
### マッピング ###
library(QuasR)
library(Rsamtools)

In [ ]:
time_s <- proc.time()                  #計算時間計測用(マッピング開始時間の記録)
out <- qAlign(in_f1, in_f2, splicedAlignment=F, alignmentParameter=param)
time_e <- proc.time()                  #計算時間計測用(マッピング終了時間の記録)

alignment files missing - need to:
    create alignment index for the genome
    create 2 genomic alignment(s)
Creating an Rbowtie index for /var/folders/8g/l6tj9lds0z3b15_dc_wgvy9w0000gn/T//RtmpXytANd/file2ffd47ce0c52.fa
Finished creating index
Testing the compute nodes...OK
Loading QuasR on the compute nodes...OK
Available cores:


nodeNames
kyamadaPro.local 
               1 


Performing genomic alignments for 2 samples. See progress in the log file:
/Users/yamada/RNA-seq/【R】トランスクリプトーム解析/chapter2/mapping/human/QuasR_log_2ffd6c28cea1.txt


In [ ]:
### QCレポート ###
out_f <- sub(".bam", "_QC.pdf", out@alignments[,1])
qQCReport(out, pdfFilename=out_f)

#### マッピングの結果を見てみる

In [ ]:
alignmentStats(out)
time_e - time_s

### カウントデータの取得

 install RMariaDB  
 https://github.com/r-dbi/RMariaDB

conda install -c r r-rmariadb

TranscriptDB形式のtxdbオブジェクトを取得する

In [26]:
library(GenomicFeatures)

In [22]:
param1 <- "hg38"
param2 <- "ensGene"
txdb <- makeTxDbFromUCSC(genome=param1, tablename=param2)

ERROR: Error in .tablename2track(tablename, session): UCSC table "ensGene" is not supported


エラー  
https://support.bioconductor.org/p/107839/

In [29]:
ucscGenomes()[ , "db"]

[1] hg38        hg19        hg18        hg17        hg16        vicPac2    
  [7] vicPac1     dasNov3     papAnu4     papAnu2     papHam1     bisBis1    
 [13] panPan2     panPan1     aptMan1     otoGar3     felCat9     felCat8    
 [19] felCat5     felCat4     felCat3     panTro6     panTro5     panTro4    
 [25] panTro3     panTro2     panTro1     criGri1     criGriChoV2 criGriChoV1
 [31] manPen1     bosTau9     bosTau8     bosTau7     bosTau6     bosTau4    
 [37] bosTau3     bosTau2     macFas5     canFam3     canFam2     canFam1    
 [43] turTru2     loxAfr3     musFur1     thaSir1     nomLeu3     nomLeu2    
 [49] nomLeu1     aquChr2     rhiRox1     gorGor5     gorGor4     gorGor3    
 [55] chlSab2     cavPor3     eriEur2     eriEur1     equCab3     equCab2    
 [61] equCab1     dipOrd1     galVar1     triMan1     calJac3     calJac1    
 [67] pteVam1     myoLuc2     balAcu1     mm10        mm9         mm8        
 [73] mm7         micMur2     micMur1     hetGla2     hetGla1     monDom5    
 [79] monDom4     monDom1     ponAbe2     ponAbe3     ailMel1     susScr11   
 [85] susScr3     susScr2     ochPri3     ochPri2     ornAna2     ornAna1    
 [91] nasLar1     oryCun2     rn6         rn5         rn4         rn3        
 [97] rheMac8     rheMac3     rheMac2     proCap1     oviAri4     oviAri3    
[103] oviAri1     sorAra2     sorAra1     choHof1     speTri2     saiBol1    
[109] tarSyr2     tarSyr1     sarHar1     echTel2     echTel1     tupBel1    
[115] macEug2     cerSim1     xenLae2     allMis1     gadMor1     melUnd1    
[121] galGal6     galGal5     galGal4     galGal3     galGal2     latCha1    
[127] calMil1     fr3         fr2         fr1         petMar3     petMar2    
[133] petMar1     anoCar2     anoCar1     oryLat2     geoFor1     oreNil2    
[139] chrPic1     gasAcu1     tetNig2     tetNig1     nanPar1     melGal5    
[145] melGal1     xenTro9     xenTro7     xenTro3     xenTro2     xenTro1    
[151] taeGut2     taeGut1     danRer11    danRer10    danRer7     danRer6    
[157] danRer5     danRer4     danRer3     ci3         ci2         ci1        
[163] braFlo1     strPur2     strPur1     apiMel2     apiMel1     anoGam3    
[169] anoGam1     droAna2     droAna1     droEre1     droGri1     dm6        
[175] dm3         dm2         dm1         droMoj2     droMoj1     droPer1    
[181] dp3         dp2         droSec1     droSim1     droVir2     droVir1    
[187] droYak2     droYak1     caePb2      caePb1      cb3         cb1        
[193] ce11        ce10        ce6         ce4         ce2         caeJap1    
[199] caeRem3     caeRem2     priPac1     aplCal1     sacCer3     sacCer2    
[205] sacCer1     eboVir3    
233 Levels:  ailMel1 allMis1 anoCar1 anoCar2 anoGam1 anoGam3 apiMel1 ... xenTro9

In [21]:
supportedUCSCtables()

tablename,track,subtrack
<chr>,<fct>,<chr>
knownGene,UCSC Genes,NA
knownGeneOld8,Old UCSC Genes,NA
knownGeneOld7,Old UCSC Genes,NA
knownGeneOld6,Old UCSC Genes,NA
knownGeneOld4,Old UCSC Genes,NA
knownGeneOld3,Old UCSC Genes,NA
ccdsGene,CCDS,NA
refGene,NCBI RefSeq,UCSC RefSeq
xenoRefGene,Other RefSeq,NA


In [32]:
supportedUCSCtables("hg38")

tablename,track,subtrack
<chr>,<fct>,<chr>
knownGeneOld8,Old UCSC Genes,NA
knownGeneOld7,Old UCSC Genes,NA
knownGeneOld6,Old UCSC Genes,NA
knownGeneOld4,Old UCSC Genes,NA
knownGeneOld3,Old UCSC Genes,NA
ccdsGene,CCDS,NA
refGene,NCBI RefSeq,UCSC RefSeq
xenoRefGene,Other RefSeq,NA
sibGene,SIB Genes,NA


ensGeneが見つからない

In [33]:
supportedUCSCtables("hg19")

tablename,track,subtrack
<chr>,<fct>,<chr>
knownGene,UCSC Genes,NA
knownGeneOld8,Old UCSC Genes,NA
knownGeneOld7,Old UCSC Genes,NA
knownGeneOld6,Old UCSC Genes,NA
knownGeneOld4,Old UCSC Genes,NA
knownGeneOld3,Old UCSC Genes,NA
ccdsGene,CCDS,NA
refGene,NCBI RefSeq,UCSC RefSeq
xenoRefGene,Other RefSeq,NA


In [34]:
txdb <- makeTxDbFromUCSC(genome="hg38", tablename="knownGeneOld8")

ERROR: Error in .tablename2track(tablename, session): UCSC table "knownGeneOld8" does not exist for genome "hg38", sorry


In [35]:
txdb <- makeTxDbFromUCSC(genome="hg38", tablename="geneid")

Download the geneid table ... OK
Extract the 'transcripts' data frame ... OK
Extract the 'splicings' data frame ... OK
Download and preprocess the 'chrominfo' data frame ... OK
Prepare the 'metadata' data frame ... OK
Make the TxDb object ... OK


In [36]:
txdb

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: UCSC
# Genome: hg38
# Organism: Homo sapiens
# Taxonomy ID: 9606
# UCSC Table: geneid
# UCSC Track: Geneid Genes
# Resource URL: http://genome.ucsc.edu/
# Type of Gene ID: no gene ids
# Full dataset: yes
# miRBase build ID: NA
# transcript_nrow: 35910
# exon_nrow: 218915
# cds_nrow: 218915
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2019-08-27 19:27:09 +0900 (Tue, 27 Aug 2019)
# GenomicFeatures version at creation time: 1.34.1
# RSQLite version at creation time: 2.1.1
# DBSCHEMAVERSION: 1.2

In [ ]:
#カウント
count <- qCount(out, txdb, reportLevel="gene")
head(count)
dim(count)

In [ ]:
#保存
out_f <- "SRA000299_ensgene.txt"
tmp <- cbind(rownames(count), count)
write.table(tmp, out_f, sep="\t", append=F, quote=F, row.names=F)

### 第3章